In [1]:
from mpl_toolkits.mplot3d import Axes3D
from numpy import *
import matplotlib; 
from matplotlib.pyplot import *
#import seaborn as sns
#sns.set()
%matplotlib notebook 

In [2]:
def Q(f, xs, ys):
    X, Y = meshgrid(xs, ys)
    fx = vectorize(lambda x, y: f([x, y])[0])
    fy = vectorize(lambda x, y: f([x, y])[1])
    return X, Y, fx(X, Y), fy(X, Y)

In [3]:
def f(xy):
    x, y = xy
    return array([-y, x])

In [4]:
figure()
x = y = linspace(-1.0, 1.0, 20)
gca().set_aspect(1.0); grid(True)
quiver(*Q(f, x, y)) 

<IPython.core.display.Javascript object>

In [5]:
figure()
x = y = linspace(-1.0, 1.0, 20)
gca().set_aspect(1.0); grid(True)
streamplot(*Q(f, x, y), color="k") 

<IPython.core.display.Javascript object>

Pendulum
---------

$$
  \begin{array}{lll}
  \dot{\theta} &=& \omega \\
  \dot{\omega} &=& - (b/m\ell^2) \omega -(g /\ell) \sin \theta  
  \end{array}
  $$


In [6]:
m=1.0; b=0.0; l=1.0; g=9.81
def f(theta_d_theta):
    theta, d_theta = theta_d_theta
    J = m * l * l
    d2_theta  = - b / J * d_theta 
    d2_theta += - g / l * sin(theta)
    return array([d_theta, d2_theta])
        

In [7]:
figure()
theta = linspace(-1.5 * pi, 1.5 * pi, 100)
d_theta = linspace(-5.0, 5.0, 100)
grid(True)
xticks([-pi, 0, pi], [r"$-\pi$", "$0$", r"$\pi$"])
streamplot(*Q(f, theta, d_theta), color="k") 

<IPython.core.display.Javascript object>

Vinograd
---------

In [8]:
def f(xy):
    x, y = xy
    q = x**2 + y**2 * (1 + (x**2 + y**2)**2) 
    dx = (x**2 * (y - x) + y**5) / q
    dy = y**2 * (y - 2*x) / q
    return array([dx, dy])

figure()
x = y = linspace(-1.0, 1.0, 1000)
streamplot(*Q(f, x, y), color="k") 
xticks([-1, 0, 1])
plot([0], [0], "k.", ms=10.0)

<IPython.core.display.Javascript object>

Euler Scheme
------------

(fixed step + explicit)

In [9]:
def solve_ivp_euler_explicit(f, t0, x0, dt, t_f):
    ts, xs = [t0], [x0]
    while ts[-1] < t_f:
        t, x = ts[-1], xs[-1]
        t_next, x_next = t + dt, x + dt * f(x)
        ts.append(t_next); xs.append(x_next)
    return (array(ts), array(xs).T)

In [10]:
def solve_ivp_euler_implicit(f, t0, x0, dt, t_f, itermax = 100):
    ts, xs = [t0], [x0]
    while ts[-1] < t_f:
        t, x = ts[-1], xs[-1]
        t_next, x_next0 = t + dt, x + dt * f(x)
        x_next1 = x + dt*f(x_next0)
        iter = 1;
        while linalg.norm(x_next1-x_next0)/linalg.norm(x_next0)>0.01 and iter<itermax:
            x_next0 = x_next1;
            x_next1 = x + dt*f(x_next0)
            iter += 1
        ts.append(t_next); xs.append(x_next1)
    return (array(ts), array(xs).T)

In [11]:
def solve_ivp_heun(f, t0, x0, dt, t_f):
    ts, xs = [t0], [x0]
    while ts[-1] < t_f:
        t, x = ts[-1], xs[-1]
        t_next, x_next = t + dt, x + dt/2 * (f(x) + f(x+dt*f(x)) )
        ts.append(t_next); xs.append(x_next)
    return (array(ts), array(xs).T)

In [12]:
def solve_ivp_trapezes(f, t0, x0, dt, t_f, itermax = 100):
    ts, xs = [t0], [x0]
    while ts[-1] < t_f:
        t, x = ts[-1], xs[-1]
        t_next, x_next0 = t + dt, x + dt/2 * (f(x) + f(x+dt*f(x)))
        x_next1 = x + dt/2 * (f(x) + f(x+dt*f(x_next0)))
        iter = 1;
        while linalg.norm(x_next1-x_next0)/linalg.norm(x_next0)>0.01 and iter<itermax:
            x_next0 = x_next1;
            x_next1 = x + dt/2 * (f(x) + f(x+dt*f(x_next0)))
            iter += 1
        ts.append(t_next); xs.append(x_next1)
    return (array(ts), array(xs).T)

In [13]:
def f(xy):
    x, y = xy
    return array([-y, x])

t0, x0 = 0.0, array([-1.0, 0.0])
dt, tf = 0.01, 10.0
t, x = solve_ivp_euler_explicit(f, t0, x0, dt, tf)

figure()
plot(x[0],x[1])
grid(True)

t, x = solve_ivp_euler_implicit(f, t0, x0, dt, tf)

figure()
plot(x[0],x[1])
grid(True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
finfo(float).eps

2.220446049250313e-16

Euler Scheme with variable step
------------

In [15]:
def solve_ivp_euler_explicit_variable_step(f, t0, x0, t_f, dtmin = 1e-16, dtmax = 0.1, atol = 1e-3):
    dt = dtmax/10; # initial integration step
    ts, xs = [t0], [x0]  # storage variables
    t = t0
    ti = 0  # internal time keeping track of time since latest storage point : must remain below dtmax
    x = x0
    while ts[-1] < t_f:
        while ti < dtmax:
            t_next, ti_next, x_next = t + dt, ti + dt, x + dt * f(x)
            x_back = x_next - dt * f(x_next)
            ratio_abs_error = atol / (linalg.norm(x_back-x)/2)
            dt = 0.9 * dt * sqrt(ratio_abs_error)
            if dt < dtmin:
                raise ValueError("Time step below minimum")
            elif dt > dtmax/2:
                dt = dtmax/2
            t, ti, x = t_next, ti_next, x_next
        dt2DT = dtmax - ti # time left to dtmax
        t_next, ti_next, x_next = t + dt2DT, 0, x + dt2DT * f(x)
        ts.append(t_next); xs.append(x_next)
        t, ti, x = t_next, ti_next, x_next
    return (array(ts), array(xs).T)


In [16]:
# Robertson stiff system 
def f(xabc):
    xa, xb, xc = xabc
    return array([-0.04 * xb + 1e4 * xb * xc, 0.04 * xa - 1e4 * xb * xc - 3e7 * xb**2,  3e7 * xb**2])

t0, tf, x0 = 0.0, 5.0, array([1.0, 0.0, 0.0])

t, x = solve_ivp_euler_explicit_variable_step(f, t0, x0, tf, dtmin = 1e-20, atol = 1)

figure()
plot(t, x[0])
plot(t, x[1])
plot(t, x[2])
grid(True)

ValueError: Time step below minimum

In [ ]:
t, x = solve_ivp_euler_explicit_variable_step(f, t0, x0, tf)

figure()
plot(t, x[0])
plot(t, x[1])
grid(True)

Already-made solvers
-----------------------------
Documentation: [`solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) (Scipy)

In [ ]:
from scipy.integrate import solve_ivp

In [ ]:
def fun(t, y):
    x1, x2 = y
    return array([-x2, x1])
t_span = [0.0, 2*pi]
y0 = [1.0, 0.0]
result = solve_ivp(fun=fun, t_span=t_span, y0=y0)

The `result` is a dictionary-like object (a “bunch”).
Its fields:

    - t : array, time points, shape (n_points,),

    - y : array, values of the solution at t, shape (n, n_points),

    …

The solver may apply to systems that are not time-invariant
$$
  \dot{y} = f(t, y)
  $$
  The `t` argument in the definition of `fun` is mandatory, even if the
returned value doesn’t depend on it (time-invariant system).



In [ ]:
r_t = result["t"]
x_1 = result["y"][0]
x_2 = result["y"][1]

figure()
t = linspace(0, 2*pi, 1000)
plot(t, cos(t), "k--")
plot(t, sin(t), "k--")
bold = {"lw": 2.0, "ms": 10.0}
plot(r_t, x_1, ".-", label="$x_1(t)$", **bold)
plot(r_t, x_2, ".-", label="$x_2(t)$", **bold)
xlabel("$t$"); grid(); legend()

The step size $t_{n+1}-t_n$ is variable and automatically selected by the algorithm.

The solver shall meet the user specification, but should select the largest step size to do so to minimize the number of computations.

Optionally, you can specify a `max_step` (default: $+\infty$).

We generally want to control the (local) error $e(t)$:\
the difference between the numerical solution and the exact one.

-   `atol` is the **absolute tolerance** (default: $10^{-6}$),

-   `rtol` is the **relative tolerance** (default: $10^{-3}$).

The solver ensures (approximately) that at each step:
$$
  |e(t)| \leq \mathrm{atol} + \mathrm{rtol} \times |x(t)|
  $$



In [ ]:
options = {
    # at least 20 data points
    "max_step": 2*pi / 20, 
    # standard absolute tolerance
    "atol"    : 1e-6,        
    # very large relative tolerance
    "rtol"    : 1e9 
}

In [ ]:
result = solve_ivp(
    fun=fun, t_span=t_span, y0=y0, 
    **options
)
r_t = result["t"]
x_1 = result["y"][0]
x_2 = result["y"][1]

figure()
t = linspace(0, 2*pi, 20)
plot(t, cos(t), "k--")
plot(t, sin(t), "k--")
bold = {"lw": 2.0, "ms": 10.0}
plot(r_t, x_1, ".-", label="$x_1(t)$", **bold)
plot(r_t, x_2, ".-", label="$x_2(t)$", **bold)
xlabel("$t$"); grid(); legend()

Using a small `max_step` is usually the wrong way to “get more data
points” since this will trigger many (potentially expensive) evaluations
of `fun`.

Instead, use dense outputs: the solver may return\ the discrete data result["t"] and result["y"] and an approximate solution result["sol"] as a function of t with little extra computations.

In [ ]:
options = {
    "dense_output": True
}

In [ ]:
result = solve_ivp(
    fun=fun, t_span=t_span, y0=y0, 
    **options
)
r_t = result["t"]
x_1 = result["y"][0]
x_2 = result["y"][1]
sol = result["sol"]

figure()
t = linspace(0, 2*pi, 1000)
plot(t, cos(t), "k--")
plot(t, sin(t), "k--")
bold = {"lw": 2.0, "ms": 10.0}
plot(t, sol(t)[0], "-", label="$x_1(t)$", **bold)
plot(t, sol(t)[1], "-", label="$x_2(t)$", **bold)
plot(r_t, x_1, ".", color="C0", **bold)
plot(r_t, x_2, ".", color="C1", **bold)
xlabel("$t$"); grid(); legend()

Proie/prédateur 
-----------------------------

In [ ]:
alpha = 2/3; beta = 4/3; delta = 1; gamma = 1;
def fun(t, y):
    x1, x2 = y
    return array([x1*(alpha-beta*x2), x2*(delta*x1-gamma)])


options = {
    "max_step": 0.01, 
    "atol"    : 1e-6,        
    "rtol"    : 1e9,
    "dense_output" : True
}

t_span = [0.0, 20.0]
y0 = [0.9, 0.9]
result = solve_ivp(fun=fun, t_span=t_span, y0=y0,**options)

r_t = result["t"]
x_1 = result["y"][0]
x_2 = result["y"][1]




In [ ]:
figure()
plot(r_t, x_1, label="Proies")
plot(r_t, x_2, label="Prédateurs")
xlabel("$t$"); grid(); legend()

figure()
plot(x_1,x_2)
xlabel("Proies")
ylabel("Prédateurs")
grid(); 


In [ ]:
def f(y):
    x1, x2 = y
    return array([x1*(alpha-beta*x2), x2*(delta*x1-gamma)])

figure()
x1 = linspace(0, 2, 100)
x2 = linspace(0, 1.5, 100)
grid(True)
streamplot(*Q(f, x1, x2), color="k") 

t0, x0 = 0.0, array([0.9, 0.9])
dt, tf = 0.01, 100
t, x = solve_ivp_euler_explicit(f, t0, x0, dt, tf)
H_explicit = delta*x[0]-gamma*log(x[0]) + beta*x[1]-alpha*log(x[1])

figure()
plot(x[0],x[1])
grid(True)

t, x = solve_ivp_euler_implicit(f, t0, x0, dt, tf)
H_implicit = delta*x[0]-gamma*log(x[0]) + beta*x[1]-alpha*log(x[1])

figure()
plot(x[0],x[1])
grid(True)

t, x = solve_ivp_heun(f, t0, x0, dt, tf)
H_heun = delta*x[0]-gamma*log(x[0]) + beta*x[1]-alpha*log(x[1])

figure()
plot(x[0],x[1])
grid(True)

t, x = solve_ivp_trapezes(f, t0, x0, dt, tf)
H_trapezes = delta*x[0]-gamma*log(x[0]) + beta*x[1]-alpha*log(x[1])

figure()
plot(x[0],x[1])
grid(True)

figure()
plot(t,H_explicit,label = "Euler explicite")
plot(t,H_implicit,label = "Euler implicite")
plot(t,H_heun,label = "Heun")
plot(t,H_trapezes,label = "Trapèzes")
legend()

figure()
plot(t,H_heun,label = "Heun")
plot(t,H_trapezes,label = "Trapèzes")
legend()

In [ ]:
k = 100;
def f_mod(y):
    x1, x2 = y
    H = delta*x1-gamma*log(x1) + beta*x2-alpha*log(x2)
    return array([x1*(alpha-beta*x2)-k*(H-H0)*(delta-gamma/x1), x2*(delta*x1-gamma)-k*(H-H0)*(beta-alpha/x2)])

t0, x0 = 0.0, array([0.9, 0.9])
H0 = delta*x0[0]-gamma*log(x0[0]) + beta*x0[1]-alpha*log(x0[1])
dt, tf = 0.01, 100
t, x = solve_ivp_euler_explicit(f_mod, t0, x0, dt, tf)
H_explicit_mod = delta*x[0]-gamma*log(x[0]) + beta*x[1]-alpha*log(x[1])

figure()
plot(x[0],x[1])
grid(True)

figure()
#plot(t,H_explicit,label = "Euler explicite")
plot(t,H_explicit_mod,label = "Euler explicite modifié")
legend()



Attracteur de Lorenz
-----------------------

In [ ]:
sigma = 10; rho = 28; beta = 8/3;
def fun(t, y):
    x1, x2, x3 = y
    return array([sigma*(x2-x1), rho*x1-x2-x1*x3, x1*x2-beta*x3])

options = {
    "max_step": 0.01, 
    "atol"    : 1e-6,        
    "rtol"    : 1e9,
    "dense_output" : True
}

t_span = [0.0, 50.0]
y0 = [1.0, 0.0 ,0.0]
result = solve_ivp(fun=fun, t_span=t_span, y0=y0,**options)

r_t = result["t"]
x_1 = result["y"][0]
x_2 = result["y"][1]
x_3 = result["y"][2]

#figure()
#bold = {"lw": 2.0, "ms": 10.0}
#plot(r_t, x_1, ".-", label="$x_1(t)$", **bold)
#plot(r_t, x_2, ".-", label="$x_2(t)$", **bold)
#plot(r_t, x_3, ".-", label="$x_3(t)$", **bold)
#xlabel("$t$"); grid(); legend()


In [ ]:

fig = figure()
ax = fig.gca(projection='3d')

ax.plot(x_1,x_2,x_3)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')


Two-body problem
=====================

In [ ]:
e = 0.01
q0 = array([1-e,0.0])
p0 = array([0.0,sqrt((1+e)/(1-e))])
t0, tf, dt = 0.0 , 50.0, 0.01

In [ ]:
def solve_ivp_euler_symplectic(fp, fq, t0, q0, p0, dt, tf):
    ts, qs, ps = array([t0]), array([q0]), array([p0])
    while ts[-1] < tf:
        t, q, p = ts[-1], qs[-1], ps[-1]
        t_next, q_next = t + dt, q + dt*fq(p)
        p_next = p + dt*fp(q_next)
        ts = vstack([ts,t_next])
        qs = vstack([qs,q_next])
        ps = vstack([ps,p_next])
    return (ts, qs, ps)

In [ ]:
def fq(p):
    return p

def fp(q):
    q1,q2 = q
    return array([-q1/power(q1**2+q2**2,3/2),-q2/power(q1**2+q2**2,3/2)])

def fun(qp):
    q1,q2,p1,p2 = qp
    return concatenate([fq([p1,p2]),fp([q1,q2])])

In [ ]:
t, q, p = solve_ivp_euler_symplectic(fp,fq, t0, q0, p0, dt, tf)

t, qp = solve_ivp_euler_explicit(fun, t0, concatenate([q0,p0]), dt, tf)


def fun_ivp(t,qp):
    q1,q2,p1,p2 = qp
    return concatenate([fq([p1,p2]),fp([q1,q2])])

result = solve_ivp(fun=fun_ivp, t_span=[t0,tf], y0=concatenate([q0,p0]),**options)

figure()
plot(q[...,0], q[...,1])
plot([0.0],[0.0],'.')
grid(True)
title('Euler symplectique')

figure()
plot(qp[0],qp[1])
plot([0.0],[0.0],'.')
grid(True)
title('Euler explicite')

figure()
plot(result["y"][0],result["y"][1])
plot([0.0],[0.0],'.')
grid(True)
title('Solver Python')


Solar system
============

In [ ]:
N = 6;
m_n = 1.9891e30; # mass unit (sun)
d_n = 149597870; # length unit (sun-earth) (km)
t_n = 3600*24; # one day on earth (s)
#G = 6.67384e-11*m_n*t_n**2/d_n**3; 
G = 2.95912208286e-4;
#e = 0.01671022

#0: Sun
#1 : Jupiter
#2 : Saturn
#3 : Uranus
#4 : Neptune
#5 : Pluto

m0 = 1.00000597682; # correction of sun mass to take into account the inner planets
m1 = 0.000954786104043;
m2 = 0.000285583733151;
m3 = 0.0000437273164546;
m4 = 0.0000517759138449;
m5 = 1/(1.3e8);
M = array([m0,m1,m2,m3,m4,m5]);

q0 = array([[0.0,0.0,0.0],
            [-3.5023653,-3.8169847,-1.5507963],
            [9.0755314,-3.0458353,-1.6483708],
            [8.3101420,-16.2901086,-7.2521278],
            [11.4707666,-25.7294829,-10.8169456],
            [-15.5387357,-25.2225594,-3.1902382]
           ])

q0 = q0.reshape(3*N)

v0 = array([[0.0,0.0,0.0],
            [0.00565429,-0.00412490,-0.00190589],
            [0.00168318,0.00483525,0.00192462],
            [0.00354178,0.00137102,0.00055029],
            [0.00288930,0.00114527,0.00039677],
            [0.00276725,-0.00170702,-0.00136504]
           ])

v0 = v0.reshape(3*N)

t0, tf, dt = 0.0 , 10000, 1.0

In [ ]:
def fq(v):
    return v

def fv(q):
    dynv = zeros(3*N);
    qbis = q.reshape((N,3))  # each line is the position of one entity
    for j in range(N):
        qbis_others = delete(qbis,j,0) # removes jth line
        M_others = delete(M,j).reshape((N-1,1))
        dist_vect = -(qbis_others - qbis[j])
        dist = sqrt(dist_vect.T[0]**2+dist_vect.T[1]**2+dist_vect.T[2]**2).reshape((N-1,1)) # computation of the norm
        dynv[3*j:3*j+3] = -G*sum(M_others*dist_vect/dist**3,0)
    return dynv
        


In [ ]:
t, q, p = solve_ivp_euler_symplectic(fq, fv, t0, q0, v0, dt, tf)

fig = figure()
ax = fig.gca(projection='3d')
for j in range(N):
    ax.plot(q.T[3*j], q.T[3*j+1], q.T[3*j+2])
    


In [ ]:
G

In [ ]:
tt2

In [ ]:
zeros(3)

In [ ]:
tt[1::2]

In [ ]:
Z =zeros((2,N))


In [ ]:
Z[...,0]

In [ ]:
ttt = array([1,2,3])*ones((2,3))

In [ ]:
ttt[1]